# Random Forests
Es un algoritmo predictivo, entrando en la categoria de algoritmos de Machine Learning orientados de aprendizaje por union.

En particular este modelo consiste en combinacion de modelos independientes que pueden ser similares o diferentes para resolver un problema de prediccion en particular.

En si conciste en utilizar muchos arboles para clasificar y predicir, luego agarramos los datos de cada uno de estos y calculamos la media.

#### Metodos de esamblaje
* **Metodo del promedio:** se crean varios modelos similares e independientes y se hace un promedio de las predicciones de cada modelo.
* **Metodo de impulso:** reducir el sesgo estimador combinado construyendolo de forma secuecion de los estimadores base. Asi conseguimos un modelo mas robusto a partir de varios mas debiles.

Este modelo realmente se puede combinar a cualquier modelo de regresion o clasificacion. Es solo el promedio de muchos modelos.

#### Caracteristicas:
* No necesita ningun metodo de validacion cruzada, ya que se utiliza el metodo de **`bagging`**
    * Con `n` observaciones en el dataset de entrenamiento `T` y `m` variables, decidimos hacer crecer `S` arboles en nuestro bosque. Cada uno sera creado por un dataset de entrenamiento separado.
    * Las `n` observaciones para cada dataset se toma aleatoriamente con remplazo del dataset original.
    * Cada dataset puede obtener observaciones duplicadas y algunas no aparecer nunca como entrenamiento.
    * Estos dataset se llaman muestras de `bootstrap` o `bolsas`
    
    
#### Algoritmo:
 * Tomamos una muestra aleatoria simple de tamanio n con remplazo.
 * Tomamos una muestra aleatoria simple de variables predictoras sin remplazo.
 * Construimos un arbol de regresion con los predictores elegidos en 2 y sin podar el arbol.
 * Clasificamos las observaciones fuera de la bolsa con dicho arbol y almacenamos el valor o la clase asignada para cada una.
 * Repetimos los 4 anteriores pasos un largo de veces para tener el bosque de arboles.
 * La prediccion final es el promedio de las observaciones de todos los arboles para clasificacion, la clase con mayoria de votos en el conjunto de arboles.

# Random Forest para Regresion
Vamos a utilizar el mismo dataset que trabajamos con anterioridad

In [16]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor

In [4]:
data = pd.read_csv('../python-ml-course-master/datasets/boston/Boston.csv')
columns = data.columns.values.tolist()
predicts = columns[:13]
target = columns[13]
x = data[predicts]
y = data[target]
regtree = DecisionTreeRegressor(min_samples_leaf=10,min_samples_split=30,random_state=0)
regtree.fit(x,y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=10, min_samples_split=30,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=0, splitter='best')

In [11]:
data['reg_tree'] = regtree.predict(data[predicts])

In [5]:
from sklearn.ensemble import RandomForestRegressor

In [32]:
# n_jobs = Especificar la paralizacion de las tareas para hacer en el ordenador
# n_estimadors = Cuantos arboles vamos a utilizar
#El modelo va a hacer un muestro como quiera (el modelo se encarga)
forest = RandomForestRegressor(n_jobs=2,n_estimators=100,oob_score=True)
forest.fit(x,y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=2, oob_score=True,
                      random_state=None, verbose=0, warm_start=False)

In [33]:
data['forest_pred'] = forest.oob_prediction_

In [34]:
data[['medv','reg_tree','forest_pred']]

medv   reg_tree  forest_pred
0    24.0  22.840000    27.385714
1    21.6  22.840000    22.217949
2    34.7  35.247826    34.320000
3    33.4  35.247826    35.041935
4    36.2  35.247826    34.546341
..    ...        ...          ...
501  22.4  22.840000    23.986364
502  20.6  20.624138    19.128571
503  23.9  28.978261    26.720513
504  22.0  31.170000    26.115217
505  11.9  20.624138    20.711905

[506 rows x 3 columns]

In [35]:
data['pred1_err'] = (data['reg_tree'] - data['medv'])**2
data['pred2_err'] = (data['forest_pred'] - data['medv'])**2

In [36]:
print('promedio de error cuadrado de regTree: ',np.mean(data['pred1_err']))
print('promedio de error cuadrado de regForest: ',np.mean(data['pred2_err']))

promedio de error cuadrado de regTree:  12.46593406540346
promedio de error cuadrado de regForest:  10.08180178334945


In [37]:
# Es similar al coeficiente de determinacion que utilizabamos en regresion lineal iba entre 0 y 1
#mientras mas cerca de 1 mejor
forest.oob_score_

0.8805750439542778

# Random Forest para Regresion
Vamos a utilizar el mismo dataset de las flores

In [41]:
df = pd.read_csv('../python-ml-course-master/datasets/iris/iris.csv')
columns = df.columns.values.tolist()
predicts = columns[:4]
target = columns[4]
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion='entropy', min_samples_split=20,random_state=99)
tree.fit(df[predicts],df[target])
preds = tree.predict(df[predicts])
pd.crosstab(df[target],preds,rownames=["Actual"],colnames=['Predictions'])

Predictions  setosa  versicolor  virginica
Actual                                    
setosa           50           0          0
versicolor        0          47          3
virginica         0           0         50

In [42]:
from sklearn.ensemble import RandomForestClassifier

In [49]:
forest = RandomForestClassifier(n_jobs=2,n_estimators=100, oob_score=True)
forest.fit(df[predicts],df[target])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
                       oob_score=True, random_state=None, verbose=0,
                       warm_start=False)

In [50]:
forest.oob_score_

0.96